# RL Coursework 1


**Total number of marks:** 10 

**What to submit:** Completed workbook (.ipynb file)

**Where to submit:** CM50270 Moodle page

The lab exercises in this workbook will determine 10% of your final mark for CM50270. 

You need to work on this coursework on your own. You are welcome to discuss it with your classmates but you must write your own code.  

Remember to save your work regularly.

You must comply with the universities plagiarism guidelines: http://www.bath.ac.uk/library/help/infoguides/plagiarism.html


## Introduction


In this coursework, you will implement the Value Iteration algorithm to compute an optimal policy for three different (but related) Markov Decision Processes. For your reference, the pseudo-code for the algorithm is reproduced below from the textbook (Reinforcement Learning, Sutton & Barto, 1998). 

<img src="images/value_iteration.png" style="width: 800px;"/>

Please note the following about the pseudo-code: The set $\mathcal{S}$ contains all non-terminal states, whereas $\mathcal{S}^+$ is the set of all states (terminal and non-terminal). The reward $r = r(s, a, s')$ is the expected immediate reward on transition from state $s$ to the next state $s'$ under action $a$. 

<img src="images/bombs and gold numbers.png" style="width: 300px;" align="left" caption="Figure 1"/>

The three problems you will solve use variants of the gridworld environment shown on the left. You should be familiar with this environment from the lectures and from your previous lab exercise. The grid squares in the figure are numbered as shown. In all three problems, the following is true: 

**Actions available:** The agent has four possible actions in each grid square. These are _west_, _north_, _south_, and _east_. If the direction of movement is blocked by a wall (for example, if the agent executes action south at grid square 1), the agent remains in the same grid square. 

**Collecting gold:** On its first arrival at a grid square that contains gold, the agent collects the gold. In order to collect the gold, the agent needs to transition into the grid square (containing the gold) from a different grid square. 

**Hitting the bomb:** On arrival at a grid square that contains the bomb, the agent activates the bomb. 

** Terminal states:** The game terminates when all gold is collected or when the bomb is activated. In Exercises 1 and 2, you can define terminal states to be grid squares 18 and 23. In Exercise 3, you will need to define terminal state(s) differently.


### Instructions ###
Set parameter $\theta$ to 1 to the power of -10. You can express that as `1e-10` in Python. 

Set all initial state values $V(s)$ to zero.

Do not use discounting (that is, set $\gamma=1$).

Use the following reward function: $-1$ for each navigation action (including when the action results in hitting the wall), an additional $+10$ for collecting each piece of gold, and an additional $-10$ for activating the bomb. For example, the immediate reward for transitioning into a square with gold is $-1 + 10 = +9$. 


## Exercise 1: Deterministic environment (3 marks)

In this first exercise, the agent is able to move in the intended direction with certainty. For example, if it executes action _north_ in grid square 0, it will transition to grid square 5 with probability 1. In other words, we have a deterministic environment. 

Compute the optimal policy using Value Iteration. 

Your method needs to output two one-dimensional numpy arrays with names `policy` and `v`. Both arrays should have a length of 25, with the element at index $i$ representing grid cell $i$ (see figure above). Both arrays should be callable in the "solution cell" below! 

The array `policy` should be a numpy array of strings that specifies an optimal action at each grid location. Please use the abbreviations "n", "e", "s", and "w" for the four actions. As an example, policy at index 0 needs to give "n", if _north_ is an optimal action in cell 0. The policy for a terminal state can be any value (direction). If there are multiple optimal actions from a state, any optimal action will be considered as a correct answer. 

The array `v` should be an array of floats that contains the expected return at each grid square (that is, the state value).

In [1]:
############################################################ RL CWK 1 ###########################################################


"""
The program is to solve the grid world problems using the value iteration algorithm as described in the book Reinforcement
Learning: An Introduction by Richard S. Sutton and Andrew G. Barto.

The program has two classes, the first is GridWorld, this solves the first two problems by implementingg a single gold bar
version, with two different update rules, both a stoichastic and determanistic update.  In reality the stoichastic update
can be usesd in place of the determanistic update by setting the action probability to one, however this is really inefficient
hence I made a determanistic update function.

The second class is GridWorldTwoGolds, this inherits from the first, meaing that only a few parameters and one methed had to be
overwritten, simplifying code.

I have written all of the program in the first box with the relevant testing mechanisms in the respective boxes.

Finally terminal states need not an optimal policy (Though one could be calculated greedily), consequently I have represented
these as either 'g' for a gold bar or 'b' for a bomb.  With respect to the third problem the two initial gold bars are also
placed as g as they are "portals" to the next state which does have a policy.
"""


############################################################ Imports ############################################################


import numpy as np


############################################################ Classes ############################################################


class GridWorld():
    """
    This is a class representing a grid world for a value iteration example.
    """


    def __init__(self, goldIndex, bombIndex):
        """
        Class constructor.  The grid world is represented by a 1D array of length 25 - consequently an agents position is a
        single integer index.  I have consequently created a look-up table, move, which stores the index an agent will end up at
        for a given movement.  Although hard coding is bad practice, this nonetheless creates an computationally efficient way
        to perform the Value Iteration algorithm.  The table is structured as follows, with N representing at which index the
        agent will end up at if it moves North etc.  The table is cast as int as these are indexes.

         Index 0   Index 1        Index 25
        [[N,E,S,W],[N,E,S,W], ... [N,E,S,W]]

        The array reward is an array representing the external reward gained for each move, this has a value of -10 at index 18
        to represent the bomb, and a value of 10 at index 23 to represent the gold.  The array values is where the value function
        data is stored, this is intiially all set to zero as per the algorithm.  To speed up calculations, both have been set to
        a data type of float to prevent the values being cast every operation.
        """

        self.move = np.array([[ 5, 1, 0, 0],[ 6, 2, 1, 0],[ 7, 3, 2, 1],[ 8, 4, 3, 2],[ 9, 4, 4, 3],
                              [10, 6, 0, 5],[11, 7, 1, 5],[12, 8, 2, 6],[13, 9, 3, 7],[14, 9, 4, 8],
                              [15,11, 5,10],[16,12, 6,10],[17,13, 7,11],[18,14, 8,12],[19,14, 9,13],
                              [20,16,10,15],[21,17,11,15],[22,18,12,16],[23,19,13,17],[24,19,14,18],
                              [20,21,15,20],[21,22,16,20],[22,23,17,21],[23,24,18,22],[24,24,19,23]], dtype = int)

        self.goldIndex = goldIndex
        self.bombIndex = bombIndex
        # Sets the position of the gold and bomb in the world.

        self.values = np.zeros(25, dtype = float)
        self.reward = np.zeros(25, dtype = float)
        self.policy = np.empty(25, dtype = str)

        self.reward[self.goldIndex] = 10
        self.reward[self.bombIndex] = -10
        # Sets the rewards for the gold and the bomb.

        self.goldIndexList = [self.goldIndex]
        self.terminalIndexList = [self.goldIndex,self.bombIndex]
        # By having these lists instead of hard coding, it is possible to have multiple terminal states and gold terminal
        # states, this is to allow easy inheritance for the GridWorldTwoGolds class.


    def updateStateDet(self, stateIndex):
        """
        This function performs a single value update for a given state.  This is the determanistic version of the function.
        The function alsso returns the state Delta, basically |(v-V(s))| in the algorithm.
        """

        oldValue = self.values[stateIndex] # Records the current value in the value array.
        newValue = 0.0

        for i in range(4): # Iterates through N, E, S, W actions.
            newIndex = self.move[stateIndex,i] # Gets the index of the next state given the action.
            tempValue = self.reward[newIndex]-1+self.values[newIndex]
            # p(s',r|s,a)[r+gammaV(s')] : p = 1 as determanistic, gamma = 1 => [r+V(s')]
            if tempValue > newValue: newValue = tempValue # Used to find the max value for each of the actions.

        self.values[stateIndex] = newValue # Sets the old value in the array to the max value of the actions.

        return abs(oldValue-newValue)


    def updateStateSto(self, stateIndex, pAction):
        """
        This function performs a single value update for a given state.  This is the stoichastic version of the function.
        Although the determanistic version is simply the stoichastic version with pAction set to 1, this is very ineffecient
        hence a seperate function has been provided. pAction is the probability of the requested action happening.
        """

        pRanMove = (1-pAction)/4 # This is the probability of an individual random move happening.
        pDesMove = pAction+pRanMove # This is the probability of the desired move happening.

        oldValue = self.values[stateIndex] # Records the current value in the value array.
        newValue = 0.0

        for i in range(4): # Iterates through N, E, S, W actions.
            tempValue = 0.0 # Sigma(p(s',r|s,a)[r+gammaV(s')])  : gamma = 1

            for j in range(4): # Again iterates through N, E, S, W to calculate the expected value for an action.
                newIndex = self.move[stateIndex,j] # Gets the index of the next state given the action.
                if i == j: # This represents the desired moves.
                    expectation = pDesMove*(self.reward[newIndex]-1+self.values[newIndex])
                    tempValue += expectation # p(s',r|s,a)[r+gammaV(s')] : gamma = 1
                else: # This represents undesired moves.
                    expectation = pRanMove*(self.reward[newIndex]-1+self.values[newIndex])
                    tempValue += expectation # p(s',r|s,a)[r+gammaV(s')] : gamma = 1
            if tempValue > newValue: newValue = tempValue # Used to find the max value for each of the actions.

        self.values[stateIndex] = newValue # Sets the old value in the array to the max value of the actions.

        return abs(oldValue-newValue)


    def updateAllStates(self, theta, determanistic = False, pAction = 0.0):
        """
        This function updates the entire value array given theta, the prescision limit.  Also takes an argument to choose whether
        to use the determanistic version to speed things up for worlds with pAction = 1.0 .
        """

        delta = 1 # Sets delta to one so the program passes into the while loop.
        convergance = 0

        while delta > theta: # This loops until delta is smaller than theta.
            delta = 0 # Sets delta to 0.

            for i in range(len(self.values)): # Loops over each state.
                if i not in self.terminalIndexList: # Forces the terminal states to stay at zero.
                    if determanistic == True:
                        stateDelta = self.updateStateDet(i)
                    else:
                        stateDelta = self.updateStateSto(i, pAction)
                    delta = max(delta, stateDelta)
            convergance += 1

        print("Converged in "+str(convergance)+" iterations.")


    def updatePolicy(self):
        """
        This function greedily updates the policy given the values array.
        """

        for i in self.goldIndexList:
            self.values[i] = max(self.values)+1
        # For the algorithm the terminal states are fixed at zero however this means the following algorithm fails, therefore
        # the gold value is temporarily set to the max value in the values array plus 1.

        directionLookUp = ["n","e","s","w"]

        for i in range(len(self.values)): # loops over each state.
            if i not in self.terminalIndexList: # Keeps the terminal states empty.
                direction = 0
                value = 0.0
                for j in range(4): # Loops over each direction.
                    searchIndex = self.move[i, j] # Finds the index in values to look at using the move look up table.
                    searchValue = self.values[searchIndex] # Finds the value at aforementioned index.
                    if searchValue > value: # This section finds the direction of the largest value.
                        value = searchValue
                        direction = j
                self.policy[i] = directionLookUp[direction]
            elif i in self.goldIndexList: # Puts a g in the gold states.
                self.policy[i] = "g"
            else: # Puts a b in bomb states.
                self.policy[i] = "b"

        for i in self.goldIndexList:
            self.values[i] = 0.0 # Resets the terminal state back to zero.


    def printPolicy(self):
        """
        Prints the policy in the representative grid, makes checking simple.
        """
        count = 0
        world = ""
        for i in range(5):
            line = ""
            for j in range(5):
                if self.policy[count] == "n":
                    line += "↑ "
                elif self.policy[count] == "e":
                    line += "→ "
                elif self.policy[count] == "s":
                    line += "↓ "
                elif self.policy[count] == "w":
                    line += "← "
                else:
                    line += self.policy[count] + " "
                count += 1
            world = line + "\n" + world
        print("\n"+world)
    
    def outputArrays(self):
        """
        Returns both the policy and the values arrays, in the specified format with N's instead of g's and b's.
        """
        outputPolicy = np.copy(self.policy)
        
        for i in range(len(outputPolicy)):
            if outputPolicy[i] not in ["n","e","s","w"]:
                outputPolicy[i] = "n"
        
        return self.values, outputPolicy



class GridWorldTwoGolds(GridWorld):
    """
    Inherits from the GridWorld class but ovverides move, values, reward and policy to account for having two peices of gold.
    Essentially picking  up one of the gold peices acts as a portal to a "new" gridworld with only one peice of gold, in order
    to do this there must be 75 states or three worlds, and move must be updated to reflect this such that it has this "portalling"
    functionllity.
    """


    def __init__(self, goldIndexList, bombIndex):
        """
        Class Constructor.  As this inherits from GridWorld, fundementaly only self.move, self.values, self.reward,
        self.policy, self.goldIndexList and self.terminalIndexList must be modified.  All methods remain unmodified.
        """

        self.goldIndexList = goldIndexList + [goldIndexList[0]+50, goldIndexList[1]+25]
        # The indexs of the gold bars.
        self.portalIndexList = [goldIndexList[0]+25, goldIndexList[1]+50]
        # The index of the portal exits.
        self.terminalIndexList = [bombIndex, bombIndex+25, bombIndex+50] + self.goldIndexList
        # Location of all terminal states, three bombs plus four golds.

        self.move = np.array([[ 5,  1,  0,  0],[ 6,  2,  1,  0],[ 7,  3,  2,  1],[ 8,  4,  3,  2],[ 9,  4,  4,  3],
                             [10,  6,  0,  5],[11,  7,  1,  5],[12,  8,  2,  6],[13,  9,  3,  7],[14,  9,  4,  8],
                             [15, 11,  5, 10],[16, 12,  6, 10],[17, 13,  7, 11],[18, 14,  8, 12],[19, 14,  9, 13],
                             [20, 16, 10, 15],[21, 17, 11, 15],[22, 18, 12, 16],[23, 19, 13, 17],[24, 19, 14, 18],
                             [20, 21, 15, 20],[21, 22, 16, 20],[22, 23, 17, 21],[23, 24, 18, 22],[24, 24, 19, 23],
                             [30, 26, 25, 25],[31, 27, 26, 25],[32, 28, 27, 26],[33, 29, 28, 27],[34, 29, 29, 28],
                             [35, 31, 25, 30],[36, 32, 26, 30],[37, 33, 27, 31],[38, 34, 28, 32],[39, 34, 29, 33],
                             [40, 36, 30, 35],[41, 37, 31, 35],[42, 38, 32, 36],[43, 39, 33, 37],[44, 39, 34, 38],
                             [45, 41, 35, 40],[46, 42, 36, 40],[47, 43, 37, 41],[48, 44, 38, 42],[49, 44, 39, 43],
                             [45, 46, 40, 45],[46, 47, 41, 45],[47, 48, 42, 46],[48, 49, 43, 47],[49, 49, 44, 48],
                             [55, 51, 50, 50],[56, 52, 51, 50],[57, 53, 52, 51],[58, 54, 53, 52],[59, 54, 54, 53],
                             [60, 56, 50, 55],[61, 57, 51, 55],[62, 58, 52, 56],[63, 59, 53, 57],[64, 59, 54, 58],
                             [65, 61, 55, 60],[66, 62, 56, 60],[67, 63, 57, 61],[68, 64, 58, 62],[69, 64, 59, 63],
                             [70, 66, 60, 65],[71, 67, 61, 65],[72, 68, 62, 66],[73, 69, 63, 67],[74, 69, 64, 68],
                             [70, 71, 65, 70],[71, 72, 66, 70],[72, 73, 67, 71],[73, 74, 68, 72],[74, 74, 69, 73]])
        # Updated version of the move look up table to represent the three worlds.

        self.move[self.goldIndexList[0]] = self.move[self.portalIndexList[0]]
        self.move[self.goldIndexList[1]] = self.move[self.portalIndexList[1]]
        # This modifies the lookup table to create the prortals by replacing the movements from the gold index list with
        # that in the portal index list.

        self.values = np.zeros(75, dtype = float)
        self.reward = np.zeros(75, dtype = float)
        self.policy = np.empty(75, dtype = str)
        # Initialised with length 75.

        for i in range(3):
            self.reward[self.terminalIndexList[i]] = -10
        # Adds the rewards for the bomb.

        for i in self.goldIndexList:
            self.reward[i] = 10
        # Adds the rewards for the gold bars.


    def printPolicy(self):
        """
        Overwrites the previous as this requires a different print method.
        """
        count = 0
        print()
        for i in range(3):
            world = ""
            for j in range(5):
                line = ""
                for k in range(5):
                    if self.policy[count] == "n":
                        line += "↑ "
                    elif self.policy[count] == "e":
                        line += "→ "
                    elif self.policy[count] == "s":
                        line += "↓ "
                    elif self.policy[count] == "w":
                        line += "← "
                    else:
                        line += self.policy[count] + " "
                    count += 1
                world = line + "\n" + world
            print(world)


############################################################ Testing ############################################################


gridWorldDet = GridWorld(23,18)
# Creates determanistic grid world.
gridWorldDet.updateAllStates(1e-10, determanistic = True)
# Performs the Value iteration algorithm
gridWorldDet.updatePolicy()
# Updates the policy greedily acording to the values.
gridWorldDet.printPolicy()
# Prints a nice arrow version for easy visual conformation.

v, policy = gridWorldDet.outputArrays()
# Outputs 'g' for a gold bar and 'b' for a bomb, please see line 18 for details.

print( np.flip(     v.reshape(5,5),0), "\n")
print( np.flip(policy.reshape(5,5),0), "\n")
# Printing v and policy for visual conformation.

Converged in 8 iterations.

→ → → g ← 
↑ ↑ ↑ b ↑ 
↑ ↑ ↑ → ↑ 
↑ ↑ ↑ ↑ ↑ 
↑ ↑ ↑ ↑ ↑ 

[[7. 8. 9. 0. 9.]
 [6. 7. 8. 0. 8.]
 [5. 6. 7. 6. 7.]
 [4. 5. 6. 5. 6.]
 [3. 4. 5. 4. 5.]] 

[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'n' 'n']] 



#### Check the data types of your solution! 
The following tests allow you to check whether the variables `policy` and `v` (which you should have computed _above_ this cell) have the correct data types. You can use the same tests for Exercises 2 and 3!

In [2]:
# Print the values you computed
print("This is your 'policy':")
print(policy)
print("These are your state values 'v':")
print(v)

# Check whether both policy and v are numpy arrays.
import numpy as np
assert(isinstance(policy, np.ndarray))
assert(isinstance(v, np.ndarray))

# Check correct shapes of numpy arrays.
assert(policy.shape == (25, ))
assert(v.shape == (25, ))

# Check whether the numpy arrays have the correct data types.
assert(np.issubdtype(policy.dtype, np.unicode_)) # policy.dtype should be '<U1'
assert(np.issubdtype(v.dtype, np.float64))

# Check whether all policy values are either "n", "w", "s", or "e".
assert(np.all(np.isin(policy, np.array(["n", "w", "s", "e"])))) 

# Arrays with CORRECT data types (but WRONG values!) would be, for example:
# policy = np.array(["n", "w", "s", "w", "e", "n", "w", "s", "w", "e", 
#                    "n", "w", "s", "w", "e", "n", "w", "s", "w", "e", 
#                    "n", "w", "s", "w", "e"])
# v = np.random.rand(25)
# DO NOT UNCOMMENT THE PREVIOUS lines... otherwise they will overwrite the arrays that you computed!

This is your 'policy':
['n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'n' 'e' 'n' 'n' 'n' 'n'
 'n' 'n' 'e' 'e' 'e' 'n' 'w']
These are your state values 'v':
[3. 4. 5. 4. 5. 4. 5. 6. 5. 6. 5. 6. 7. 6. 7. 6. 7. 8. 0. 8. 7. 8. 9. 0.
 9.]


In [3]:
# DO NOT DELETE THIS CELL. 
# Your code for Exercise 1 is tested here. 


## Exercise 2: Stochastic environment (4 marks)

In this second exercise, the agent is not always able to execute its actions as intended. With probability 0.8, it moves in the intended direction. With probability 0.2, it moves in a random direction. For example, from grid square 0, if the agent executes action _north_, with probability 0.8, the action will work as intended. But with probability 0.2, the agent's motor control system will move in a random direction (including north). For example, with probability 0.05, it will try to move west (where it will be blocked by the wall and hence remain in grid square 0). Notice that the total probability of moving to square 5 (as intended) is 0.8 + 0.05 = 0.85.
 
Compute the optimal policy using Value Iteration.

Your value iteration method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercise 1.

In [4]:
gridWorldSto = GridWorld(23,18)
# Creates stoichastic grid world.
gridWorldSto.updateAllStates(1e-10, pAction = 0.8)
# Performs the Value iteration algorithm
gridWorldSto.updatePolicy()
# Updates the policy greedily acording to the values.
gridWorldSto.printPolicy()
# Prints a nice arrow version for easy visual conformation.

v, policy = gridWorldSto.outputArrays()
# Outputs 'g' for a gold bar and 'b' for a bomb, please see line 18 in part 1 for details.

print( np.flip(     v.reshape(5,5),0), "\n")
print( np.flip(policy.reshape(5,5),0), "\n")
# Printing v and policy for visual conformation.

Converged in 30 iterations.

→ → → g ← 
↑ ↑ ↑ b ↑ 
↑ ↑ ↑ → ↑ 
↑ ↑ ↑ → ↑ 
↑ ↑ ↑ ↑ ↑ 

[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]] 

[['e' 'e' 'e' 'n' 'w']
 ['n' 'n' 'n' 'n' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'e' 'n']
 ['n' 'n' 'n' 'n' 'n']] 



In [5]:
# Do not delete this cell. Your code for Exercise 2 is tested here.

## Exercise 3: Stochastic environment with two pieces of gold (3 marks)

<img src="images/bomb and two gold.png" style="width: 300px;" align="left" caption="Figure 1"/> In this third exercise, the environment is identical to the environment in Exercise 2 with the following exception: there is an additional piece of gold on grid square 12. Recall from earlier instructions that the terminal state is reached only when _all_ gold is collected or when the bomb is activated.

Compute the optimal policy using Value Iteration. 

Hint: You will need to change your state representation. 

Your method should output two one-dimensional numpy arrays with names `policy` and `v`, as in Exercises 1 and 2. These arrays should specify the expected return and an optimal policy at the corresponding grid sqaure **before any gold is collected or a bomb is activated.** 

In [6]:
gridWorldTwo = GridWorldTwoGolds([12,23],18)
# Creates stoichastic two gold grid world.
gridWorldTwo.updateAllStates(1e-10, pAction = 0.8)
# Performs the Value iteration algorithm
gridWorldTwo.updatePolicy()
# Updates the policy greedily acording to the values.
gridWorldTwo.printPolicy()
# Prints a nice arrow version for easy visual conformation.

v, policy = gridWorldTwo.outputArrays()
# Outputs 'g' for a gold bar and 'b' for a bomb, please see line 18 in part 1 for details.

print( np.flip(     v[:25  ].reshape(5,5),0), "\n")
print( np.flip(     v[25:50].reshape(5,5),0), "\n")
print( np.flip(     v[50:  ].reshape(5,5),0), "\n")
print( np.flip(policy[:25  ].reshape(5,5),0), "\n")
print( np.flip(policy[25:50].reshape(5,5),0), "\n")
print( np.flip(policy[50:  ].reshape(5,5),0), "\n")
# Printing v and policy for visual conformation.

Converged in 30 iterations.

→ → → g ← 
→ ↓ ↓ b ↑ 
→ → g ← ← 
→ ↑ ↑ ← ← 
↑ ↑ ↑ ↑ ← 

→ → → g ← 
↑ ↑ ↑ b ↑ 
↑ ↑ ↑ → ↑ 
↑ ↑ ↑ → ↑ 
↑ ↑ ↑ ↑ ↑ 

↓ ↓ ↓ ← ↓ 
→ ↓ ↓ b ↓ 
→ → g ← ← 
→ ↑ ↑ ← ← 
↑ ↑ ↑ ↑ ← 

[[6.27185527 7.4483022  8.69750488 0.         8.69690012]
 [6.27225747 7.38830369 7.80429043 0.         6.5442022 ]
 [7.29987383 8.60242493 0.         7.68979128 6.49253977]
 [6.18412144 7.36032106 8.59757988 7.30328591 6.08760162]
 [5.07297558 6.18349601 7.2879906  6.12946745 5.01603046]] 

[[6.04169329 7.28756636 8.61359951 0.         8.69262311]
 [4.86185111 5.99087587 6.37082431 0.         6.46721593]
 [3.67550938 4.69621388 4.99441863 3.2189158  5.10250988]
 [2.48699534 3.40945989 3.66922967 2.64122933 3.78610115]
 [1.35979208 2.19733672 2.42878751 1.57272161 2.55202451]] 

[[5.01464931 6.08004088 6.36762762 4.28547547 3.17436436]
 [6.12844981 7.3022647  7.68349462 0.         4.28547547]
 [7.28739724 8.59725345 0.         7.68349462 6.36762762]
 [6.1787892  7.35540702 8.59725345 7.302264

In [7]:
# Do not delete this cell. Your code for Exercise 3 is tested here.


# IMPORTANT: How to submit.

If any of the following instructions is not clear, please ask your tutors well ahead of the submission deadline.

### Before you submit
- Restart the kernel (_Kernel $\rightarrow$ Restart & Run All_) and make sure that you can run all cells from top to bottom without any errors.
- Make sure that the entire notebook runs through in less than 5 minutes.
- Make sure that every test cell has access to the correct arrays and make sure that the data types of these arrays are correct.
- Make sure that your code is written in Python 3 (and not in Python 2!). You can check the Python version of the current session in the top-right corner below the Python logo.

### Submission file
- Please upload to Moodle a single Jupyter notebook file called "rl2_value_iteration.ipynb". Do __not__ compress/zip your Jupyter notebook file.
- Do not include ANY identifying information. Marking is anonymous.
